In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import os

## prepare data

In [2]:
# def load_train_label():
#     train_data_path = "/Users/xyli1905/Projects/Datasets/Kaggle/QuoraToxicDetect/train.csv"
#     data = pd.read_csv(train_data_path)
#     data_label = np.array(data["target"])
#     return data_label

# def loadVocab():
#     # load existed vocabulary
#     vocab_path =  "/Users/xyli1905/Projects/NLP/detoxic/data_proc/vocab.txt"
#     with open(vocab_path, 'r') as f:
#         vocab = [word.split("\n")[0] for word in f.readlines()]
#     return vocab

# def loadTraintoken():
#     # load tokenized training data
#     train_token_path = "/Users/xyli1905/Projects/NLP/detoxic/data_proc/traintoken.pkl"
#     with open(train_token_path, 'rb') as f:
#         traintoken = pickle.load(f)
#     return traintoken

# def loadPretrainedWeight():
#     # load pretrained embedding
#     weight_data_path = "/Users/xyli1905/Projects/NLP/detoxic/data_proc/pretrained_weight.npy"
#     pretrained_weight = np.load(weight_data_path)
#     return pretrained_weight

In [3]:
# # define parameters of training data
# sentence_cutoff = 60
# embedding_dim = 300

# def prepare_training_data():
#     # load data
#     vocab = loadVocab()
#     vocab_len = len(vocab)
#     idx_nothing = vocab_len
    
#     # load train token, builing train tensor with cutoff 60
#     word_to_idx = {word: i for i, word in enumerate(vocab)}
    
#     train_token = loadTraintoken()
#     train_idx = np.full((len(train_token),sentence_cutoff), idx_nothing)
#     for i, sentence in enumerate(train_token):
#         for j, word in enumerate(sentence):
#             if j >= sentence_cutoff:
#                 break
#             train_idx[i,j] = word_to_idx[word]
#     train_tensor = torch.tensor(train_idx, dtype=torch.long)
            
#     # load training label
#     labels = load_train_label()
#     train_label = torch.from_numpy(labels).view(-1,1)
    
#     # set up pretrained embedding
#     Pweight = loadPretrainedWeight()
#     ## insert vector representing nothing at the end of pretrained embedding
#     weight_nothing = np.zeros(embedding_dim)
#     pretrained_weight = torch.from_numpy(np.vstack([Pweight, weight_nothing]))
    
#     # concatenate train_tensor & train_label to get full train data
#     train_data = torch.cat((train_tensor, train_label), 1)

#     return vocab_len, train_data, pretrained_weight

In [97]:
# vocab_len, train_data, pretrained_weight = prepare_training_data()

In [2]:
# # loading preprocessed data
# with open('../data_proc/processed_data/train_mat.pkl', 'rb') as f:
#     # train_mat.pkl is np.ndarray
#     train_data = torch.from_numpy(pickle.load(f))
# with open('../data_proc/processed_data/pretrained_weight.pkl', 'rb') as f:
#     # pretrained_weight is np.ndarray
#     pretrained_weight = torch.from_numpy(pickle.load(f))

In [3]:
# sentence_cutoff = train_data.shape[1] - 1
# vocab_len = pretrained_weight.shape[0] - 2
# embedding_dim = pretrained_weight.shape[1]

In [4]:
# print(type(train_data), type(pretrained_weight))
# print(train_data.shape, pretrained_weight.shape)
# print(sentence_cutoff, vocab_len, embedding_dim)

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([1306122, 61]) torch.Size([52309, 300])
60 52307 300


In [8]:
# ckpt_dir = "/Users/xyli1905/Projects/NLP/detoxic/checkpoints"

In [7]:
# valid_num = 50000

## define train class

In [6]:
# from torch.utils.data import DataLoader
# import time

In [9]:
# class Train:
#     def __init__(self, model, max_epoch=5):
#         self._chkp_dir = ckpt_dir
#         # parameter
#         self._max_epoch = max_epoch
#         self._batch_size = 64
        
#         # data processing for training
#         #self._dataset = train_data
#         self._dataset_train = DataLoader(train_data[:-valid_num,:], batch_size=self._batch_size,
#                                          shuffle=True, drop_last=True)

#         # define model
#         self._model = model
#         self._lossfun = torch.nn.CrossEntropyLoss()
#         self._optimizer = torch.optim.SGD(self._model.parameters(), lr=0.01)
        
#         # train model
#         self._train()
        
#         # update training flag
#         if self._model.trained:
#             self._model.training_times += 1
#         else:
#             self._model.trained = True
#             self._model.training_times = 1
    
#     def _train(self):
#         # may initialize info of training here
#         # e.g. self._total_step
#         # ...
        
#         for i_epoch in range(self._max_epoch):
#             epoch_time_start = time.time()
            
#             # train epoch
#             print("Start epoch %d / %d, \t at %s" % (i_epoch+1, self._max_epoch, time.asctime()))
#             self._train_epoch(i_epoch)
        
#             # save model after each epoch here
#             if (i_epoch != 0) and (i_epoch % 5 == 0 or i_epoch == self._max_epoch):
#                 idx = self._model.save()
#                 self._save_opt(idx)
            
#             # training time for each epoch
#             time_cost = time.time() - epoch_time_start
#             print("End of epoch %d / %d \t Time taken: %d sec (or % d min)" % \
#                   (i_epoch+1, self._max_epoch, time_cost, time_cost / 60.))
            
#             # may updata learning rate here
#             # if i_epoch > ...

#     def _train_epoch(self, i_epoch):

#         for i_train_batch, train_batch in enumerate(self._dataset_train):
            
#             # forward model
#             y_pred = self._model.forward(train_batch[:, :-1])
#             LRloss = self._lossfun(y_pred, train_batch[:, -1])
            
#             # backprop
#             self._optimizer.zero_grad()
#             LRloss.backward()
#             self._optimizer.step()
            
#     def _save_opt(self, idx):
#         '''
#         idx is from model.save, equal to model.training_times
#         save optimizer parameters
#         '''
#         fname = 'opt_{}_{}_id.pth'.format(self._model.name, str(idx))
#         save_path = os.path.join(self._chkp_dir, fname)
#         torch.save(self._optimizer.state_dict(), save_path)
#         print("optimizer for %s saved at %s" % (self._model.name, save_path))
    
#     def _load_opt(self, idx=-1):
#         '''
#         load optimizer with idx,
#         if idx=-1 load the one with max idx
#         '''
#         if idx == -1:
#             idx_num = 0
#             for file in os.listdir(self._chkp_dir):
#                 if file.startswith("opt"):
#                     idx_num = max(idx_num, int(file.split('_')[2]))
#         else:
#             idx_num = idx
#         #print("loading epoch %d ..." % idx_num)
#         fname = 'opt_{}_{}_id.pth'.format(self._model.name, str(idx_num))
#         file_path = os.path.join(self._chkp_dir, fname)
#         self.load_state_dict(torch.load(file_path))
#         print("optimizer for %s loaded from %s" % (self._model.name, file_path))

### test train codes

In [12]:
# seq = torch.tensor([[1,2,4],[3,5,1]],dtype=torch.long)
# print(seq, seq.shape[0])

# TW = Variable(torch.randn(10), requires_grad=True)
# print(TW)

# TE = torch.tensor([[[1,2,3,4,5],[5,4,3,2,1],[2,1,3,4,5]],[[1,2,3,4,5],[5,4,3,2,1],[2,1,3,4,5]]], dtype=torch.float)
# print(TE, TE.shape)

# print(TW[seq])

# def weighted_embed(x, s):
#     output = torch.zeros((s.shape[0], 1, x.shape[2]))
#     ww = TW[s].view(-1, 1, 3)
#     for i in range(s.shape[0]):
#         output[i] = torch.mm(ww[i], x[i])
#     return output

# out = weighted_embed(TE, seq)
# print(out)

## define LR models

In [10]:
# ### this is the Bag of Word LR model
# ###
# class LogisticRegressionBoW(torch.nn.Module):
#     def __init__(self, trainable_emb=False):
#         super(LogisticRegressionBoW, self).__init__()
#         self.name = "LRBoW"
#         self.trained = False
#         self.training_times = 0
#         self._chkp_dir = ckpt_dir
#         self._save_mark = 0
        
#         # define parameters
#         self.vocab_size = vocab_len
#         #self.hidden_size = 60
#         self.output_size = 2 # two labels 0,1
        
#         # define layers
#         self.W = torch.nn.Parameter(torch.randn(self.vocab_size+1, self.output_size),requires_grad=True)
#         self.b = torch.nn.Parameter(torch.zeros(self.output_size),requires_grad=True)
#         #self.linear = torch.nn.Linear(self.hidden_size, self.output_size)
#         #self.sigmoid = torch.nn.Sigmoid()
#         #
#         # initialize weights of layers
#         #self.init_weight()
        
# #     def init_weight():
#         # with other layers defaultly initialized
    
#     def weighted_words(self, seq):
#         '''
#         note idx_seq must be a torch tensor of dtype torch.long
#         N is batch size, 60 is the limit of sentence len
#         below, W[idx_seq]:(N, 60), output:(N, 1, 300)
#         '''
#         output = torch.zeros((seq.shape[0], self.output_size))
#         for i in range(seq.shape[0]):
#             idxseq = seq[i]
#             output[i] = torch.sum(self.W[idxseq],0) + self.b
#         return output
            
#     def forward(self, idx_seq):
#         '''
#         note idx_seq must be a torch tensor of dtype torch.long
#         N is batch size, 60 is the limit of sentence len
#         '''
#         out = self.weighted_words(idx_seq) # idx_seq:(N, 60), out:(N, 2)
#         #out = self.linear(X).view(-1,self.output_size)
#         return out
    
#     def predict(self, idx_seq):
#         # 0: sincere; 1: toxic
#         y = F.softmax(self.forward(idx_seq), dim=1)
#         y_pred = torch.argmax(y, dim=1)
#         return y_pred
    
#     def save(self):
#         '''
#         save trained parameters
#         '''
#         fname = 'net_LRBoW_{}_id.pth'.format(str(self._save_mark))
#         self._save_mark += 1
#         save_path = os.path.join(self._chkp_dir, fname)
#         torch.save(self.state_dict(), save_path)
#         print("LRBoW saved at %s" % (save_path))
#         return self._save_mark - 1
        
#     def load(self, idx=-1):
#         '''
#         load trained model with idx,
#         if idx=-1 load the one with max idx
#         '''
#         if idx == -1:
#             idx_num = 0
#             for file in os.listdir(self._chkp_dir):
#                 if file.startswith("net"):
#                     idx_num = max(idx_num, int(file.split('_')[2]))
#         else:
#             idx_num = idx
#         #print("loading epoch %d ..." % epoch_num)
#         fname = 'net_LRBoW_{}_id.pth'.format(str(idx_num))
#         self._save_mark = idx_num
#         file_path = os.path.join(self._chkp_dir, fname)
#         self.load_state_dict(torch.load(file_path))
#         print("LRBoW loaded from %s" % (file_path))

In [11]:
#test = LogisticRegressionBoW()

In [12]:
# ### this old LR model connot compute grad for W for some unknown reason, discarded for now
# ###
# class LogisticRegressionEmbBoW(torch.nn.Module):
#     def __init__(self, trainable_emb=False):
#         super(LogisticRegressionEmbBoW, self).__init__()
#         self.name = "LRembBoW"
#         self.trained = False
#         self.training_times = 0
#         self._chkp_dir = ckpt_dir
#         self._save_mark = 0
        
#         # define parameters
#         self.vocab_size = vocab_len
#         self.emb_size = embedding_dim
#         self.trainable_emb = trainable_emb
#         self.hidden_size = 64
#         self.output_size = 2 # two labels 0,1
#         ##self-defined model parameter
#         self.W = torch.nn.Parameter(torch.randn(self.vocab_size+1),requires_grad=True)
        
#         # define layers
#         self.embed = torch.nn.Embedding(self.vocab_size+1, self.emb_size)
#         self.linear1 = torch.nn.Linear(self.emb_size, self.hidden_size)
#         self.linear2 = torch.nn.Linear(self.hidden_size, self.output_size)
#         #self.sigmoid = torch.nn.Sigmoid()
        
#         # initialize weights of layers
#         self.init_weight()
        
#     def init_weight(self):
#         self.embed.weight.data.copy_(pretrained_weight)
#         self.embed.weight.requires_grad = self.trainable_emb
#         # with other layers defaultly initialized
    
#     def weighted_embed(self, x, seq):
#         '''
#         note idx_seq must be a torch tensor of dtype torch.long
#         N is batch size, 60 is the limit of sentence len
#         below, W[idx_seq]:(N, 60), output:(N, 1, 300)
#         '''
#         output = torch.zeros((seq.shape[0], 1, x.shape[2]))
#         for i in range(seq.shape[0]):
#             idxseq = seq[i:i+1]
#             output[i] = torch.mm(self.W[idxseq], x[i])
#         return output
            
#     def forward(self, idx_seq):
#         '''
#         note idx_seq must be a torch tensor of dtype torch.long
#         N is batch size, 60 is the limit of sentence len
#         '''
#         X = self.embed(idx_seq) # idx_seq:(N, 60), X:(N, 60, 300)
#         X = self.weighted_embed(X, idx_seq)
#         X = F.relu(self.linear1(X))
#         out = self.linear2(X).view(-1,self.output_size)
#         return out
    
#     def predict(self, idx_seq):
#         # 0: sincere; 1: toxic
#         y = F.softmax(self.forward(idx_seq), dim=1)
#         y_pred = torch.argmax(y, dim=1)
#         return y_pred
    
#     def save(self):
#         '''
#         save trained parameters
#         '''
#         fname = 'net_LRembBoW_{}_id.pth'.format(str(self._save_mark))
#         self._save_mark += 1
#         save_path = os.path.join(self._chkp_dir, fname)
#         torch.save(self.state_dict(), save_path)
#         print("LRembBoW saved at %s" % (save_path))
#         return self._save_mark - 1
        
#     def load(self, idx=-1):
#         '''
#         load trained model with idx,
#         if idx=-1 load the one with max idx
#         '''
#         if idx == -1:
#             idx_num = 0
#             for file in os.listdir(self._chkp_dir):
#                 if file.startswith("net"):
#                     idx_num = max(idx_num, int(file.split('_')[2]))
#         else:
#             idx_num = idx
#         #print("loading epoch %d ..." % epoch_num)
#         fname = 'net_LRembBoW_{}_id.pth'.format(str(idx_num))
#         self._save_mark = idx_num
#         file_path = os.path.join(self._chkp_dir, fname)
#         self.load_state_dict(torch.load(file_path))
#         print("LRembBoW loaded from %s" % (file_path))

In [13]:
# # note in this notebook, vocab & Pweight are global variables
# # in this new version, we set W -> (1,300) namely use embeddings to identify words & assign them weights
# class LogisticRegressionEmb(torch.nn.Module):
#     def __init__(self, trainable_emb=False):
#         super(LogisticRegressionEmb, self).__init__()
#         self.name = "LRemb"
#         self.trained = False
#         self.training_times = 0
#         self._chkp_dir = ckpt_dir
#         self._save_mark = 0
        
#         # define parameters
#         self.vocab_size = vocab_len
#         self.emb_size = embedding_dim
#         self.trainable_emb = trainable_emb
#         self.hidden_size = 64
#         self.output_size = 2 # two labels 0,1
#         ##self-defined model parameter
#         self.W = torch.nn.Parameter(torch.randn(embedding_dim, 1),requires_grad=True)
        
#         # define layers
#         self.embed = torch.nn.Embedding(self.vocab_size+1, self.emb_size)
#         self.linear1 = torch.nn.Linear(self.emb_size, self.hidden_size)
#         self.linear2 = torch.nn.Linear(self.hidden_size, self.output_size)
#         #self.sigmoid = torch.nn.Sigmoid()
        
#         # initialize weights of layers
#         self.init_weight()
        
#     def init_weight(self):
#         self.embed.weight.data.copy_(pretrained_weight)
#         self.embed.weight.requires_grad = self.trainable_emb
#         # with other layers defaultly initialized

#     def emb_selfcorr(self, x):
#         '''
#         basically do matmul(E.T, E), dim=(300, 300), kind of a self-correlation
#         '''
#         output = torch.zeros((x.shape[0], self.emb_size, self.emb_size))
#         for i in range(x.shape[0]):
#             output[i] = torch.mm(torch.t(x[i]), x[i])
#         return output
        
#     def forward(self, idx_seq):
#         '''
#         note idx_seq must be a torch tensor of dtype torch.long
#         N is batch size, 60: sentence_cutoff, 300: embedding_dim
#         input idx_seq:(N, 60)
#         '''
#         X = self.embed(idx_seq)  # emb vec X:(N, 60, 300)
#         X = self.emb_selfcorr(X) # give corr mat: (N, 300, 300)
#         X = torch.squeeze(torch.matmul(X, self.W))   # weighted sum of embeddings (N, 300)
#         X = F.relu(self.linear1(X))
#         out = self.linear2(X).view(-1,self.output_size)
#         return out
    
#     def predict(self, idx_seq):
#         '''
#         0: sincere; 1: toxic
#         '''
#         y = F.softmax(self.forward(idx_seq), dim=1)
#         y_pred = torch.argmax(y, dim=1)
#         return y_pred
    
#     def save(self):
#         '''
#         save trained parameters
#         '''
#         fname = 'net_LRemb_{}_id.pth'.format(str(self._save_mark))
#         self._save_mark += 1
#         save_path = os.path.join(self._chkp_dir, fname)
#         torch.save(self.state_dict(), save_path)
#         print("LRemb saved at %s" % (save_path))
#         return self._save_mark - 1
        
#     def load(self, idx=-1):
#         '''
#         load trained model with idx,
#         if idx=-1 load the one with max idx
#         '''
#         if idx == -1:
#             idx_num = 0
#             for file in os.listdir(self._chkp_dir):
#                 if file.startswith("net"):
#                     idx_num = max(idx_num, int(file.split('_')[2]))
#         else:
#             idx_num = idx
#         #print("loading epoch %d ..." % epoch_num)
#         fname = 'net_LRemb_{}_id.pth'.format(str(idx_num))
#         self._save_mark = idx_num
#         file_path = os.path.join(self._chkp_dir, fname)
#         self.load_state_dict(torch.load(file_path))
#         print("LRemb loaded from %s" % (file_path))

In [19]:
# valid_num

50000

In [16]:
# test = LogisticRegression()
# result = test.forward(train_tensor[0:10])
# print(result)

## run LR model

In [14]:
# model_BoW = LogisticRegressionBoW()
# model_EmbBoW = LogisticRegressionEmbBoW()
# model_Emb = LogisticRegressionEmb()
# # print(model.W[:10])
# # print(model.linear2.weight)

In [15]:
# print("model BoW param::")
# for name, param in model_BoW.named_parameters():
#     print(name, type(param.data), param.size())

# print("\nmodel EmbBoW param::")
# for name, param in model_EmbBoW.named_parameters():
#     print(name, type(param.data), param.size())
    
# print("\nmodel Emb param::")
# for name, param in model_Emb.named_parameters():
#     print(name, type(param.data), param.size())
# # list(model.parameters())
# # idx = train_tensor[4:5]
# # model.W[train_tensor[:4]]

model BoW param::
W <class 'torch.Tensor'> torch.Size([233306, 2])
b <class 'torch.Tensor'> torch.Size([2])

model EmbBoW param::
W <class 'torch.Tensor'> torch.Size([233306])
embed.weight <class 'torch.Tensor'> torch.Size([233306, 300])
linear1.weight <class 'torch.Tensor'> torch.Size([64, 300])
linear1.bias <class 'torch.Tensor'> torch.Size([64])
linear2.weight <class 'torch.Tensor'> torch.Size([2, 64])
linear2.bias <class 'torch.Tensor'> torch.Size([2])

model Emb param::
W <class 'torch.Tensor'> torch.Size([300, 1])
embed.weight <class 'torch.Tensor'> torch.Size([233306, 300])
linear1.weight <class 'torch.Tensor'> torch.Size([64, 300])
linear1.bias <class 'torch.Tensor'> torch.Size([64])
linear2.weight <class 'torch.Tensor'> torch.Size([2, 64])
linear2.bias <class 'torch.Tensor'> torch.Size([2])


In [33]:
# def evalmodel(model):
#     test_pred = model.predict(train_data[-valid_num:,:-1])
#     difftest = test_pred - torch.squeeze(train_data[-valid_num:,-1])
    
#     A = torch.sum(test_pred)
#     B = torch.sum(train_data[-valid_num:,-1])
#     C = torch.sum(torch.abs(difftest))
    
#     print("A: %d" % (float(A)))
#     print("B: %d" % (float(B)))
#     print("C: %d" % (float(C)))
#     print("F1: %.5f" % (float(A+B-C)/float(A+B)))
#     print("ACC: %s %s" % ((1. - float(C)/float(valid_num))*100., "%"))

### model 0 (BoW)

In [17]:
# print(model_BoW.trained)
# Train(model_BoW, 6)
# print(model_BoW.trained)

False
Start epoch 1 / 5, 	 at Sun Feb 17 20:08:42 2019
End of epoch 1 / 5 	 Time taken: 232 sec (or  3 min)
Start epoch 2 / 5, 	 at Sun Feb 17 20:12:34 2019
End of epoch 2 / 5 	 Time taken: 228 sec (or  3 min)
Start epoch 3 / 5, 	 at Sun Feb 17 20:16:23 2019
End of epoch 3 / 5 	 Time taken: 228 sec (or  3 min)
Start epoch 4 / 5, 	 at Sun Feb 17 20:20:12 2019
End of epoch 4 / 5 	 Time taken: 228 sec (or  3 min)
Start epoch 5 / 5, 	 at Sun Feb 17 20:24:00 2019
End of epoch 5 / 5 	 Time taken: 229 sec (or  3 min)
True


In [34]:
# evalmodel(model_BoW)

A: 2825
B: 3089
C: 4002
F1: 0.32330
ACC: 91.996 %


### model 1 (Emb)

In [40]:
# print(model_Emb.trained)
# Train(model_Emb, 6)
# print(model_Emb.trained)

False
Start epoch 1 / 6, 	 at Sun Feb 17 20:39:07 2019
End of epoch 1 / 6 	 Time taken: 350 sec (or  5 min)
Start epoch 2 / 6, 	 at Sun Feb 17 20:44:57 2019
End of epoch 2 / 6 	 Time taken: 349 sec (or  5 min)
Start epoch 3 / 6, 	 at Sun Feb 17 20:50:46 2019
End of epoch 3 / 6 	 Time taken: 349 sec (or  5 min)
Start epoch 4 / 6, 	 at Sun Feb 17 20:56:36 2019
End of epoch 4 / 6 	 Time taken: 377 sec (or  6 min)
Start epoch 5 / 6, 	 at Sun Feb 17 21:02:53 2019
End of epoch 5 / 6 	 Time taken: 373 sec (or  6 min)
Start epoch 6 / 6, 	 at Sun Feb 17 21:09:07 2019
LRemb saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/net_LRemb_0_id.pth
optimizer for LRemb saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/opt_LRemb_0_id.pth
End of epoch 6 / 6 	 Time taken: 374 sec (or  6 min)
True


In [41]:
# evalmodel(model_Emb)

A: 1759
B: 3089
C: 2294
F1: 0.52682
ACC: 95.41199999999999 %


In [43]:
# model_Emb.training_times

1

### model 2 (EmbBoW)

In [44]:
# print(model_EmbBoW.trained)
# Train(model_EmbBoW, 6)
# print(model_EmbBoW.trained)

False
Start epoch 1 / 6, 	 at Sun Feb 17 21:21:13 2019
End of epoch 1 / 6 	 Time taken: 276 sec (or  4 min)
Start epoch 2 / 6, 	 at Sun Feb 17 21:25:49 2019
End of epoch 2 / 6 	 Time taken: 269 sec (or  4 min)
Start epoch 3 / 6, 	 at Sun Feb 17 21:30:19 2019
End of epoch 3 / 6 	 Time taken: 269 sec (or  4 min)
Start epoch 4 / 6, 	 at Sun Feb 17 21:34:48 2019
End of epoch 4 / 6 	 Time taken: 295 sec (or  4 min)
Start epoch 5 / 6, 	 at Sun Feb 17 21:39:44 2019
End of epoch 5 / 6 	 Time taken: 295 sec (or  4 min)
Start epoch 6 / 6, 	 at Sun Feb 17 21:44:39 2019
LRembBoW saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/net_LRembBoW_0_id.pth
optimizer for LRembBoW saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/opt_LRembBoW_0_id.pth
End of epoch 6 / 6 	 Time taken: 294 sec (or  4 min)
True


In [46]:
# evalmodel(model_EmbBoW)

A: 1803
B: 3089
C: 2674
F1: 0.45339
ACC: 94.652 %


In [47]:
# model_EmbBoW.training_times

1

## define LSTM&GRU model

In [220]:
class GRUmodel(torch.nn.Module):
    def __init__(self):
        super(GRUmodel, self).__init__()
        self.name = "GRUmodel"
        self.trained = False
        self.training_times = 0
        self._chkp_dir = ckpt_dir
        self._save_mark = 0
        
        # define parameters
        self._vocab_size = vocab_len
        self._emb_size = embedding_dim
        self._hidden_size = 64
        self._output_size = 2 # two labels 0,1
        self._dropout_rate = 0.2
        ## below for rnn layer
        self._number_layers = 1
        self._input_size = self._emb_size # 300
        self._cell_hidden_size = self._input_size # test
        self._cell_dropout = 0 # no dropout in GRU for now
        self._bidirectional = False
        self._num_dir = 2 if self._bidirectional else 1
        self._h_state_vsize = self._number_layers*self._num_dir
        
        # define layers
        self.embed = torch.nn.Embedding(self._vocab_size+1, self._emb_size)
        self.dropout = torch.nn.Dropout(self._dropout_rate)
#         rnnL = [] # gru layer
#         for i in range(self._number_layers):
#             inp_size = self._input_size if i == 0 else self._cell_hidden_size
#             rnnL.append(torch.nn.GRU(input_size = inp_size, 
#                                      hidden_size = self._cell_hidden_size, 
#                                      batch_first = True, # (batch, seqlen, embdim)
#                                      dropout = self._cell_dropout,
#                                      bidirectional = self._bidirectional
#                                     )
#                         )
#         self.gru = torch.nn.Sequential(*rnnL)
        self.gru = torch.nn.GRU(input_size = self._input_size, 
                                hidden_size = self._cell_hidden_size, 
                                batch_first = True, # (batch, seqlen, embdim)
                                dropout = self._cell_dropout,
                                bidirectional = self._bidirectional
                               )
        self.linear1 = torch.nn.Linear(self._cell_hidden_size, self._hidden_size)
        self.linear2 = torch.nn.Linear(self._hidden_size, self._output_size)
        
        # initialize weights in layser
        self.init_weight()
    
    def init_weight(self):
        self.embed.weight.data.copy_(pretrained_weight)
        self.embed.weight.requires_grad = False
        # with other layers defaultly initialized
        
    def forward(self, idx_seq):
        X = self.embed(idx_seq) # X:(b, seqlen, embdim)
        h = Variable(torch.zeros(self._h_state_vsize, idx_seq.shape[0], self._cell_hidden_size))
        X, h = self.gru(X, h)
        X = self.dropout(X)
        X = F.relu(self.linear1(X[:,-1,:]))
        out = self.linear2(X).view(-1,self._output_size)
        return out
        
    def predict(self, idx_seq):
        '''
        0: sincere; 1: toxic
        '''
        y = F.softmax(self.forward(idx_seq), dim=1)
        y_pred = torch.argmax(y, dim=1)
        return y_pred
    
    def save(self):
        '''
        save trained parameters
        '''
        fname = 'net_GRUmodel_{}_id.pth'.format(str(self._save_mark))
        self._save_mark += 1
        save_path = os.path.join(self._chkp_dir, fname)
        torch.save(self.state_dict(), save_path)
        print("GRUmodel saved at %s" % (save_path))
        return self._save_mark - 1
        
    def load(self, idx=-1):
        '''
        load trained model with idx,
        if idx=-1 load the one with max idx
        '''
        if idx == -1:
            idx_num = 0
            for file in os.listdir(self._chkp_dir):
                if file.startswith("net"):
                    idx_num = max(idx_num, int(file.split('_')[2]))
        else:
            idx_num = idx
        #print("loading epoch %d ..." % epoch_num)
        fname = 'net_GRUmodel_{}_id.pth'.format(str(idx_num))
        self._save_mark = idx_num
        file_path = os.path.join(self._chkp_dir, fname)
        self.load_state_dict(torch.load(file_path))
        print("GRUmodel loaded from %s" % (file_path))

In [221]:
# test = GRUmodel()

In [222]:
# print(test.forward(train_data[:10,:-1]))
# for name, param in test.named_parameters():
#     if param.requires_grad:
#         print(name, param.shape)

## run GRU model

In [223]:
model_GRU = GRUmodel()

In [224]:
for name, param in model_GRU.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

gru.weight_ih_l0 torch.Size([900, 300])
gru.weight_hh_l0 torch.Size([900, 300])
gru.bias_ih_l0 torch.Size([900])
gru.bias_hh_l0 torch.Size([900])
linear1.weight torch.Size([64, 300])
linear1.bias torch.Size([64])
linear2.weight torch.Size([2, 64])
linear2.bias torch.Size([2])


In [45]:
print(model_GRU.trained)
Train(model_GRU)
print(model_GRU.trained)

### class test overwrite/inherit

In [4]:
class basetest:
    def __init__(self):
        self.name = "base"
        
    def test(self):
        print(self.name)

In [10]:
class child(basetest):
    def __init__(self):
        super(child, self).__init__()
        self.name = "child"

In [11]:
f = child()

In [12]:
f.test()

In [15]:
xtest = torch.nn.Parameter(torch.zeros(5))

In [16]:
xtest

Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)

## test on model LR classes

In [2]:
from networks.LR import BoW, EmbBoW, EmbLR
from train import Train
import os
import pickle
import torch

In [3]:
testEB = EmbBoW()

In [4]:
with open('./data_proc/processed_data/train_mat.pkl', 'rb') as f:
    # train_mat.pkl is np.ndarray
    train_data = torch.from_numpy(pickle.load(f))

In [5]:
for name, param in testEB.named_parameters():
    print(name, type(param.data), param.size())

W <class 'torch.Tensor'> torch.Size([52309])
embed.weight <class 'torch.Tensor'> torch.Size([52309, 300])
linear1.weight <class 'torch.Tensor'> torch.Size([64, 300])
linear1.bias <class 'torch.Tensor'> torch.Size([64])
linear2.weight <class 'torch.Tensor'> torch.Size([2, 64])
linear2.bias <class 'torch.Tensor'> torch.Size([2])


In [6]:
train_data.shape

torch.Size([1306122, 61])

In [8]:
valid_num = 100000
Train(model=testEB, train_data=train_data[:-valid_num,:])

Start epoch 1 / 5, 	 at Tue Feb 19 21:42:29 2019
End of epoch 1 / 5 	 Time taken: 197 sec (or  3 min)
Start epoch 2 / 5, 	 at Tue Feb 19 21:45:47 2019
Model EmbBoW saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/net_EmbBoW_2_id.pth
optimizer for EmbBoW saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/opt_EmbBoW_2_id.pth
End of epoch 2 / 5 	 Time taken: 191 sec (or  3 min)
Start epoch 3 / 5, 	 at Tue Feb 19 21:48:58 2019
End of epoch 3 / 5 	 Time taken: 195 sec (or  3 min)
Start epoch 4 / 5, 	 at Tue Feb 19 21:52:13 2019
Model EmbBoW saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/net_EmbBoW_4_id.pth
optimizer for EmbBoW saved at /Users/xyli1905/Projects/NLP/detoxic/checkpoints/opt_EmbBoW_4_id.pth
End of epoch 4 / 5 	 Time taken: 192 sec (or  3 min)
Start epoch 5 / 5, 	 at Tue Feb 19 21:55:25 2019
End of epoch 5 / 5 	 Time taken: 189 sec (or  3 min)


In [9]:
def evalmodel(model):
    test_pred = model.predict(train_data[-valid_num:,:-1])
    difftest = test_pred - torch.squeeze(train_data[-valid_num:,-1])
    
    A = torch.sum(test_pred)
    B = torch.sum(train_data[-valid_num:,-1])
    C = torch.sum(torch.abs(difftest))
    
    print("A: %d" % (float(A)))
    print("B: %d" % (float(B)))
    print("C: %d" % (float(C)))
    print("F1: %.5f" % (float(A+B-C)/float(A+B)))
    print("ACC: %s %s" % ((1. - float(C)/float(valid_num))*100., "%"))

In [10]:
evalmodel(testEB)

A: 2913
B: 6301
C: 5492
F1: 0.40395
ACC: 94.50800000000001 %
